# Demo: Transform a Point Set from SmartSPIM to CCF Space

## Overview

This notebook demonstrates how to apply the transformation results from the `RegisterToCCF` reproducible pipeline notebook to point set data initially in correspondence with the input SmartSPIM image. The notebook can be run interactively in CodeOcean.

## Inputs

1. Source point set. Source data is assumed to be in correspondence with the input SmartSPIM image used in the `RegisterToCCF` notebook.

![Point annotations on SmartSPIM input image](images/point-set-annotation.png)

2. Transform stack results from `RegisterToCCF` notebook. This is a sequence of transforms mapping from SmartSPIM sample space to CCF atlas$^1$ space produced by the SmartSPIM to CCF registration pipeline.

3. CCF atlas defining reference space for transformation.

## Outputs

1. Source point set resampled into CCF space.

## Assumptions

1. Source data initially exists in spatial alignment with the original stitched SmartSPIM image input. This can be confirmed with a spatial viewer such as ITKWidgets, Neuroglancer, or 3D Slicer.

2. The `RegisterToCCF` notebook previously ran and produced output in the `../results` directory.

## Procedure

1. Data is read in from their respective stores attached via CodeOcean's data attachment mechanism.

2. An initial translation is applied to coarsely superimpose the source point set on the target image.

3. Transforms from Elastix registration output are employed to register points into CCF space.

4. Results are written out.

## References

1. Quanxin Wang, Song-Lin Ding, Yang Li, Josh Royall, David Feng, Phil Lesnar, Nile Graddis, Maitham Naeemi, Benjamin Facer, Anh Ho, Tim Dolbeare, Brandon Blanchard, Nick Dee, Wayne Wakeman, Karla E. Hirokawa, Aaron Szafer, Susan M. Sunkin, Seung Wook Oh, Amy Bernard, John W. Phillips, Michael Hawrylycz, Christof Koch, Hongkui Zeng, Julie A. Harris, Lydia Ng,
The Allen Mouse Brain Common Coordinate Framework: A 3D Reference Atlas, Cell, Volume 181, Issue 4, 2020, Pages 936-953.e20, ISSN 0092-8674, https://doi.org/10.1016/j.cell.2020.04.007

In [1]:
import os
import json

from tqdm import tqdm
import numpy as np
import itk

itk.auto_progress(1)

In [2]:
SOURCE_IMAGE_INPUT_FILEPATH = "../data/SmartSPIM_631680_2022-09-09_13-52-33_stitched_2022-11-10_17-18-18/processed/OMEZarr/Ex_647_Em_690.zarr"
SAMPLE_ID = int(SOURCE_IMAGE_INPUT_FILEPATH.split("_")[1])
SAMPLE_CHANNEL = SOURCE_IMAGE_INPUT_FILEPATH.split("/")[-1].split(".zarr")[0]
SAMPLE_NAME = f"{SAMPLE_ID}_{SAMPLE_CHANNEL}"

# Also available at http://download.alleninstitute.org/informatics-archive/converted_mouse_ccf/average_template/
TARGET_IMAGE_INPUT_FILEPATH = (
    "../data/allen_mouse_ccf/average_template/average_template_25.nii.gz"
)

In [3]:
INPUT_POINT_DATA_FILENAME = (
    "../data/demo_point_annotation_631680/631680_Caudoputamen.mrk.json"
)

TRANSFORMS_PATH = f"../data/demo_registration_results_631680"
N_ELASTIX_STAGES = 3
ITK_TRANSFORM_FILENAME = f"{TRANSFORMS_PATH}/init-transform.hdf5"
ELASTIX_TRANSFORM_FILENAMES = [
    f"{TRANSFORMS_PATH}/elastix-transform{index}.txt"
    for index in range(N_ELASTIX_STAGES)
]

RESULTS_PATH = f"../results/{SAMPLE_NAME}"
TRANSFORMIX_POINTSET_FILE = f"{RESULTS_PATH}/transformix_input_points.txt"

# Transformix will write results to RESULTS_PATH/outputpoints.txt

## Load Points

Load a collection of points annotated in 3D Slicer and stored in 3D Slicer's point list markups JSON format.

In [4]:
with open(INPUT_POINT_DATA_FILENAME) as f:
    points_markup = json.load(f)

In [5]:
input_points = itk.PointSet[itk.F, 3].New()

for point_id, point in enumerate(points_markup["markups"][0]["controlPoints"]):
    input_points.GetPoints().InsertElement(point_id, point["position"])

print(input_points.GetPoint(0))
print(input_points)

Loading ITKPyBase... Loading ITKPyBase... Loading ITKCommon... 

itkPointF3 ([8.46357, 9.5616, 5.61258])
PointSet (0x55aba8f6dda0)
  RTTI typeinfo:   itk::PointSet<float, 3u, itk::DefaultStaticMeshTraits<float, 3u, 3u, float, float, float> >
  Reference Count: 1
  Modified Time: 4
  Debug: Off
  Object Name: 
  Observers: 
    ProgressEvent(PyCommand)
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 0
  UpdateMTime: 0
  RealTimeStamp: 0 seconds 
  Number Of Points: 53
  Requested Number Of Regions: 0
  Requested Region: -1
  Buffered Region: -1
  Maximum Number Of Regions: 1
  Point Data Container pointer: 0
  Size of Point Data Container: 0



Loading ITKCommon... Loading ITKPyUtils... Loading ITKPyUtils... 

## Apply Initial Transform

An initial transform coarsely aligns the source image into the target CCF image space. The initial transform is in ITK transform format and should be applied using ITK.

In [6]:
init_transform = itk.transformread(ITK_TRANSFORM_FILENAME)[0]
print(init_transform)

Loading ITKStatistics... Loading ITKStatistics... Loading ITKImageFilterBase... Loading ITKImageFilterBase... Loading ITKTransform... Loading ITKTransform... Loading ITKImageSources... Loading ITKImageSources... Loading ITKImageFunction... Loading ITKImageFunction... Loading ITKImageGrid... Loading ITKImageGrid... Loading ITKFFT... Loading ITKFFT... Loading ITKMesh... Loading ITKMesh... Loading ITKSpatialObjects... Loading ITKSpatialObjects... Loading ITKImageCompose... Loading ITKImageCompose... Loading ITKImageStatistics... Loading ITKImageStatistics... Loading ITKPath... Loading ITKPath... Loading ITKImageIntensity... Loading ITKImageIntensity... Loading ITKThresholding... Loading ITKThresholding... Loading ITKConvolution... Loading ITKConvolution... Loading ITKSmoothing... 

VersorRigid3DTransform (0x55aba8588e60)
  RTTI typeinfo:   itk::VersorRigid3DTransform<double>
  Reference Count: 2
  Modified Time: 647
  Debug: Off
  Object Name: 
  Observers: 
    none
  Matrix: 
    1 0 0 
    0 1 0 
    0 0 1 
  Offset: [-12.3259, -2.6141, -8.1635]
  Center: [6.6384, 9.2016, 4.176]
  Translation: [-12.3259, -2.6141, -8.1635]
  Inverse: 
    1 0 0 
    0 1 0 
    0 0 1 
  Singular: 0
  Versor: [ 0, 0, 0, 1 ]



Loading ITKSmoothing... Loading ITKDisplacementField... Loading ITKDisplacementField... Loading ITKIOTransformBase... Loading ITKIOTransformHDF5... Loading ITKIOTransformHDF5... Loading ITKIOTransformInsightLegacy... Loading ITKIOTransformInsightLegacy... Loading ITKIOTransformMatlab... Loading ITKIOTransformMatlab... Loading ITKIOTransformBase... 

In [7]:
init_points = itk.PointSet[itk.F, 3].New()
for idx in range(input_points.GetNumberOfPoints()):
    point = input_points.GetPoint(idx)
    init_points.GetPoints().InsertElement(
        idx, init_transform.TransformPoint(point)
    )
    print(f"{point} -> {init_points.GetPoint(idx)}")

itkPointF3 ([8.46357, 9.5616, 5.61258]) -> itkPointF3 ([-3.86233, 6.9475, -2.55092])
itkPointF3 ([9.42914, 9.5616, 5.32998]) -> itkPointF3 ([-2.89676, 6.9475, -2.83352])
itkPointF3 ([10.277, 9.5616, 4.3173]) -> itkPointF3 ([-2.04894, 6.9475, -3.8462])
itkPointF3 ([10.3712, 9.5616, 2.76297]) -> itkPointF3 ([-1.95474, 6.9475, -5.40053])
itkPointF3 ([8.39292, 9.5616, 4.43506]) -> itkPointF3 ([-3.93298, 6.9475, -3.72844])
itkPointF3 ([8.98168, 9.5616, 3.56369]) -> itkPointF3 ([-3.34422, 6.9475, -4.59981])
itkPointF3 ([9.12298, 9.5616, 2.50391]) -> itkPointF3 ([-3.20292, 6.9475, -5.65959])
itkPointF3 ([8.39292, 9.1584, 5.70679]) -> itkPointF3 ([-3.93298, 6.5443, -2.45671])
itkPointF3 ([9.85305, 9.1584, 4.90607]) -> itkPointF3 ([-2.47285, 6.5443, -3.25743])
itkPointF3 ([10.3476, 9.1584, 3.18688]) -> itkPointF3 ([-1.97829, 6.5443, -4.97662])
itkPointF3 ([9.61755, 9.1584, 2.1742]) -> itkPointF3 ([-2.70835, 6.5443, -5.9893])
itkPointF3 ([7.89836, 9.1584, 4.19955]) -> itkPointF3 ([-4.42754, 6.54

## Apply Transformix

Fine-grained image registration is carried out with ITKElastix and saved as sequential Elastix parameter files. Elastix transforms can be applied via `transformix` with the requirement that points to be transformed are in a specific input table format.

See the ITKElastix [PointSetAndMaskTransformation](https://github.com/InsightSoftwareConsortium/ITKElastix/blob/main/examples/ITK_Example09_PointSetAndMaskTransformation.ipynb) example for more details on applying transforms to point sets with `transformix`.

In [8]:
# Write initialized points to disk in required table format transformix to use

os.makedirs(RESULTS_PATH, exist_ok=True)
with open(TRANSFORMIX_POINTSET_FILE, "w") as f:
    f.write("point\n")
    f.write(f"{init_points.GetNumberOfPoints()}\n")

    for idx in range(init_points.GetNumberOfPoints()):
        point = init_points.GetPoint(idx)
        f.write(f"{point[0]} {point[1]} {point[2]}\n")

In [9]:
# The number of parameter maps to use is fixed during registration.
# Here we have used rigid, affine, and bspline stages in sequence.
NUM_PARAMETER_MAPS = 3

toplevel_param = itk.ParameterObject.New()
param = itk.ParameterObject.New()

for elastix_transform_filename in ELASTIX_TRANSFORM_FILENAMES:
    param.ReadParameterFile(elastix_transform_filename)
    toplevel_param.AddParameterMap(param.GetParameterMap(0))

Loading ITKOptimizers... Loading ITKOptimizers... Loading ITKImageGradient... Loading ITKImageGradient... Loading ITKImageFeature... Loading ITKImageFeature... Loading ITKFiniteDifference... Loading ITKFiniteDifference... Loading ITKRegistrationCommon... Loading ITKRegistrationCommon... Loading ITKIOImageBase... Loading ITKIOBMP... Loading ITKIOBMP... Loading ITKIOBioRad... Loading ITKIOBioRad... Loading ITKIOBruker... Loading ITKIOBruker... Loading ITKIOGDCM... Loading ITKIOGDCM... Loading ITKIOIPL... Loading ITKIOIPL... Loading ITKIOGE... Loading ITKIOGE... Loading ITKIOGIPL... Loading ITKIOGIPL... Loading ITKIOHDF5... Loading ITKIOHDF5... Loading ITKIOJPEG... Loading ITKIOJPEG... Loading ITKIOJPEG2000... Loading ITKIOJPEG2000... Loading ITKIOTIFF... Loading ITKIOTIFF... Loading ITKIOLSM... Loading ITKIOLSM... Loading ITKIOMINC... Loading ITKIOMINC... Loading ITKIOMRC... Loading ITKIOMRC... Loading ITKIOMeta... Loading ITKIOMeta... Loading ITKIONIFTI... Loading ITKIONIFTI... Loading 

In [10]:
# Load reference image (required for transformix)
average_template = itk.imread(TARGET_IMAGE_INPUT_FILEPATH, pixel_type=itk.F)

itkImageFileReaderIF3: 0.000000itkImageFileReaderIF3: 1.000000

In [ ]:
# Procedural interface of transformix filter
result_point_set = itk.transformix_pointset(
    average_template,
    toplevel_param,
    fixed_point_set_file_name=TRANSFORMIX_POINTSET_FILE,
    output_directory=RESULTS_PATH,
)

# Transformix will write results to RESULTS_PATH/outputpoints.txt

In [ ]:
print(
    "\n".join(
        [
            f"{output_point[11:18]} -> {output_point[27:35]}"
            for output_point in result_point_set
        ]
    )
)